# Peanut Butter pricing trend analysis

This notebook involves three datasets from the Nielsen datasets: purchase data, trip data, and products data. The main task is to examine the patterns of prices on top20 peannut butter products. Questions we tried to answer:    
1) How do distributions of list price and actual price look like? (for the whole category and individual product)   
2) What is average spending per household on peanut butter?     
3) How promotions influenced customer's purchase behaviors?

In [ ]:
import sqlite3

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%pylab inline

In [ ]:
!pwd

In [ ]:
cd ~/Documents/Codes/Retail_Analytics/

In [ ]:
!ls

In [ ]:
!ls ./Panel_Data/2004/Annual_Files/

In [ ]:
!head ./Panel_Data/Master_Files/Latest/retailers.tsv

In [ ]:
!wc ./Panel_Data/Master_Files/Latest/products.tsv

## combine purchase data with trip data (retailer code)

In [ ]:
purchase04 = pd.read_csv("./Panel_Data/2014/Annual_Files/purchases_2014.tsv", sep = '\t')

In [ ]:
purchase04.head(3)

In [ ]:
retailers = pd.read_csv("./Panel_Data/Master_Files/Latest/retailers.tsv", sep = '\t')

In [ ]:
len(unique(retailers["channel_type"]))

In [ ]:
unique(retailers["channel_type"])

In [ ]:
fields = ['trip_code_uc', 'household_code', 'retailer_code']
trip04_re = pd.read_csv("./Panel_Data/2014/Annual_Files/trips_2014.tsv", sep = '\t', usecols=fields)

In [ ]:
trip04_re = pd.merge(trip04_re, retailers, how='inner', on="retailer_code", sort=False)

In [ ]:
trip04_re.head(3)

In [ ]:
purchase04 = pd.merge(purchase04, trip04_re, on="trip_code_uc", how='inner',sort=False)

In [ ]:
purchase04.columns

## Select peanut butter purchase data

In [ ]:
products = pd.read_csv("./Panel_Data/Master_Files/Latest/products.tsv", sep = '\t')

In [ ]:
products.dtypes

In [ ]:
## select products belong to peanut butter category
peanut_butter = products.query('product_module_code==1421 & product_group_code == 506')

In [ ]:
peanut_butter.shape

From 2004 to 2014, there are toatl 2,748 products in peanut butter category

In [ ]:
unique(peanut_butter.product_module_descr) # check if query successful

In [ ]:
peanut_butter.set_index(['upc','upc_ver_uc'], inplace=True)

In [ ]:
peanut_butter.head(2)

In [ ]:
purchase04.set_index(['upc','upc_ver_uc'], inplace=True)

In [ ]:
purchase04.head(1)

In [ ]:
purchase04.shape

In [ ]:
peanut_04 = pd.merge(purchase04, peanut_butter, how='inner', left_index=True, right_index=True, sort=False)

In [ ]:
peanut_04.shape

(2004) peanut butter purchase file: 131,253 x 23 ;      
(2014) peanut butter purchase file: 200,924 x 23

In [ ]:
unique(peanut_04.product_module_descr)

In [ ]:
len(unique(peanut_04.channel_type))

In [ ]:
peanut_04.columns

In [ ]:
peanut_04.to_csv("./Panel_Data/2014/Annual_Files/peanut_14.csv")

## Pricing trend analysis

### 0. Load data 

In [ ]:
!pwd

In [ ]:
peanut_04 = pd.read_csv("./Panel_Data/2014/Annual_Files/peanut_14.csv")

In [ ]:
peanut_04.head(2)

In [ ]:
peanut_04.set_index(keys=['upc','upc_ver_uc'],inplace=True)

In [ ]:
peanut_04.head(2)

In [ ]:
products = pd.read_csv("./Panel_Data/Master_Files/Latest/products.tsv", sep = '\t')

In [ ]:
## select products belong to peanut butter category
peanut_butter = products.query('product_module_code==1421 & product_group_code == 506')

In [ ]:
peanut_butter.set_index(['upc','upc_ver_uc'], inplace=True)

### 1. Sum total_price_paid by upc (All channels)

In [ ]:
unique(peanut_04.size1_units)  # all products are measured in OZ

In [ ]:
unique(peanut_04.multi)  # some are multi packs

In [ ]:
peanut_04.loc[peanut_04['multi']==6].head(4)

In [ ]:
len(unique(peanut_04.index)) # there are 1046 products(2004)

In [ ]:
peanut_04_agg = peanut_04[["total_price_paid"]].groupby(level=[0, 1]).sum()
peanut_04_agg.shape

In [ ]:
peanut_04_agg.head(3)

In [ ]:
peanut_04_agg = pd.merge(peanut_04_agg, peanut_butter, how='inner', left_index=True, right_index=True, sort=False)

In [ ]:
peanut_04_agg = peanut_04_agg.sort_values(by="total_price_paid", ascending=False)
peanut_04_agg.head(5)

In [ ]:
ttl = peanut_04_agg[["total_price_paid"]].sum()
peanut_04_agg[["total_price_paid"]].iloc[1:43].sum() / ttl  

Top 28 products (out of 1046) account for 50% total sales (2004)      
Top 43 products (out of 1046) account for 50% total sales (2014)

In [ ]:
brand_count = peanut_04_agg.brand_descr.value_counts()
brand_count.head(10)

In [ ]:
df2 = pd.DataFrame([brand_count[10:].sum()], columns=["upc_counts"],index=['Others'])
df2

In [ ]:
df1 = brand_count[0:10]
df1 = df1.to_frame(name="upc_counts")
df1 = df1.append(df2)
df1

### 2. Plot distributiony of     a) list_unit_price_perOZ,    b)actual_unit_price_perOZ

In [ ]:
# compute list unit price per OZ = (total_price_paid / quantity)/(size1_amount*multi)
# compute effective unit price per OZ = [(total_price_paid - coupon_value)/ quantity] / (size1_amount*multi)

In [ ]:
peanut_04["list_unit_price_OZ"]= (peanut_04.total_price_paid / peanut_04.quantity) / (peanut_04.size1_amount * peanut_04.multi)

In [ ]:
peanut_04["actual_unit_price_OZ"]= ((peanut_04.total_price_paid - peanut_04.coupon_value) / peanut_04.quantity) /(peanut_04.size1_amount * peanut_04.multi)

In [ ]:
peanut_04.columns

In [ ]:
peanut_04.list_unit_price_OZ.mean()

In [ ]:
peanut_04.actual_unit_price_OZ.mean()

In [ ]:
bins = numpy.linspace(0, 1, 50)

In [ ]:
plt.hist(peanut_04.list_unit_price_OZ, bins)
plt.title("Distribution of price per OZ(list_price) in 2004")

In [ ]:
plt.hist(peanut_04.actual_unit_price_OZ, bins)
plt.title("Distribution of price per OZ(actual_price) in 2004")

### 3. Check Main Channel Types 

In [ ]:
channel_agg = pd.pivot_table(peanut_04, values='total_price_paid', index=["channel_type"], aggfunc='sum')

In [ ]:
tmp = channel_agg.sort_values(ascending=False)
tmp

In [ ]:
df2 = pd.DataFrame([tmp[10:].sum()], columns=["total_price_paid"],index=['Others'])

In [ ]:
df1 = tmp[0:10]
df1 = df1.to_frame()
df1 = df1.append(df2)
df1

In [ ]:
sum(df1[0:3]/df1.sum())

### 4. Check unique households in the purchase data

In [ ]:
base = len(unique(peanut_04['household_code']))
base

In [ ]:
df1.sum()/base

(2004): 30,043 households bought peanut butter, the average expenditure on peanut butter per household is 12.54;    
(2014) 45,522...... 19.10 

In [ ]:
times = peanut_04[['trip_code_uc','household_code']].groupby('household_code').count()
times.mean()

In [ ]:
times.describe()

#### Check by channel

In [ ]:
hh_channel= pd.pivot_table(peanut_04, values='household_code', index=["channel_type"], aggfunc=lambda x: len(x.unique())) ## this is the trips
tmp2 = hh_channel.sort_values(ascending=False)
tmp2.head(6)

In [ ]:
bychannel = pd.concat([hh_channel, channel_agg], axis=1)
bychannel['avg_price_paid'] = bychannel.total_price_paid / bychannel.household_code

In [ ]:
bychannel.sort_values(by='total_price_paid',ascending=False).head(6)

In [ ]:
times_channel = peanut_04[['trip_code_uc','channel_type']].groupby(['channel_type']).count()
times_channel.head(6)

In [ ]:
bychannel = pd.merge(bychannel, times_channel, left_index=True, right_index=True)
bychannel['avg_trips'] = bychannel.trip_code_uc / bychannel.household_code
bychannel = bychannel.sort_values(by='total_price_paid',ascending=False)

In [ ]:
bychannel.columns = ['total_households','total_price_paid','avg_price_paid','total_trips','avg_trips']

In [ ]:
bychannel

### 4. Check top20 products in the main channels

In [ ]:
peanut_04main = peanut_04.query('channel_type=="Grocery" or channel_type=="Discount Store" or channel_type=="Warehouse Club"')

In [ ]:
peanut_04main_agg = peanut_04main[["total_price_paid"]].groupby(level=[0, 1]).sum()
peanut_04main_agg.shape

In [ ]:
peanut_butter.shape

In [ ]:
peanut_04main_agg = pd.merge(peanut_04main_agg, peanut_butter, how='inner', left_index=True, right_index=True, sort=False)

In [ ]:
peanut_04main_agg = peanut_04main_agg.sort_values(by="total_price_paid", ascending=False)
peanut_04main_agg.head(5)

In [ ]:
top20_ls = peanut_04main_agg.head(20)
top20 = peanut_04main.loc[top20_ls.index]

In [ ]:
top20_ls.shape

### 5. Compare the prices of top20 products in different channels

In [ ]:
top20.columns

In [ ]:
top20_ls['desc']= top20_ls['upc_descr'] + " / " + top20_ls['size1_amount'].astype(str) + " " + top20_ls['size1_units'] + " / " + top20_ls['multi'].astype(str) 

In [ ]:
top20_ls[['total_price_paid','desc']]

Compute list_unit_price and actual_unit_price

In [ ]:
top20["list_unit_price"] = top20.total_price_paid / top20.quantity

In [ ]:
top20["actual_unit_price"]= ((top20.total_price_paid - top20.coupon_value) / top20.quantity)

In [ ]:
top20.list_unit_price.mean()

In [ ]:
top20.actual_unit_price.mean()

In [ ]:
top20.head(2)

In [ ]:
top20.reset_index(inplace=True)

In [ ]:
pivot = pd.pivot_table(top20, values=['list_unit_price','actual_unit_price'], columns='channel_type', index=['upc','upc_ver_uc'],aggfunc='mean')
pivot.sort_values([('list_unit_price', 'Grocery')],inplace=True)
pivot

In [ ]:
temp = pd.merge(pivot, top20_ls[["desc"]], how='left', left_index=True, right_index=True)
temp.reset_index(inplace=True)
temp['desc'].tolist()

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

In [ ]:
LABELS = temp['desc'].tolist()
pivot["list_unit_price"].plot()
plt.xticks(arange(20), LABELS, rotation=90) 
plt.title("List unit price for Peanut Butter Top20 products in different channels - 2014")

### Check price dispersion for all peanut butter products in top3 channels

In [ ]:
peanut_04main.columns

In [ ]:
peanut_04main[['coupon_value','deal_flag_uc','channel_type']].head(3)

In [ ]:
peanut04_deal = peanut_04main[['coupon_value','deal_flag_uc','channel_type']]

In [ ]:
peanut04_deal.coupon_value.max()

In [ ]:
peanut04_deal.coupon_value.min()

In [ ]:
peanut04_deal['coupon_int']= pd.cut(peanut04_deal['coupon_value'], bins=[-1, 0, 100], labels=[0, 1])

In [ ]:
peanut04_deal.head(3)

In [ ]:
def promotion(row):
   if row['deal_flag_uc'] + row['coupon_int'] == 2 :
      return 'deal+coupon'
   if row['deal_flag_uc']== 1 and row['coupon_int'] == 0 :
      return 'deal_only'
   if row['deal_flag_uc']== 0 and row['coupon_int'] == 1 :
      return 'coupon'
   return 'no_promotion'

In [ ]:
peanut04_deal['promotion'] = peanut04_deal.apply (lambda row: promotion(row),axis=1)

In [ ]:
unique(peanut04_deal.promotion)

In [ ]:
peanut04_deal.head(5)

In [ ]:
peanut04_deal.shape

In [ ]:
peanut04_deal.groupby('promotion').count()

In [ ]:
prom_Channel= peanut04_deal.groupby(['channel_type','promotion']).count()
prom_Channel

In [ ]:
tmp1 = prom_Channel.unstack(level=1)
tmp1

In [ ]:
tmp1[['coupon_int']].plot(kind='bar')
plt.title("Counts of promotion types in top3 channels for peanut butter-2004")

In [ ]:
prom_Channel[['coupon_int']].plot(kind='bar')
plt.title("Counts of promotion types in top3 channels for peanut butter")

In [ ]:
prom_pct = prom_Channel.groupby(level=0).apply(lambda x: round(x/x.sum(),2))
prom_pct

In [ ]:
tmp2 = prom_pct.unstack(level=1)
tmp2

In [ ]:
tmp2[['coupon_int']].plot(kind='bar',stacked=True)
plt.title("Percentage of different promotion types in top3 channels - 2004")

### Box plot for top20 products

In [ ]:
top20.columns

In [ ]:
top20['desc']= top20['upc_descr'] + " / " + top20['size1_amount'].astype(str) + " " + top20['size1_units'] + " / " + top20['multi'].astype(str) 

In [ ]:
top20_price = top20[['desc','actual_unit_price','list_unit_price','channel_type']]
top20_price.head(3)

In [ ]:
top20_ls[['desc','total_price_paid']]

In [ ]:
top1 = top20_price.query('desc=="JIF CRM H PLS / 18.0 OZ / 1"')
top1.boxplot(by='channel_type')

In [ ]:
def price_box(product_desc):
    query = 'desc == ' + '"' + product_desc + '"'
    tmp = top20_price.query(query)
    return tmp.boxplot(by='channel_type')

In [ ]:
price_box("JIF CRM H PLS 2P / 40.0 OZ / 2")